# Lezione 30 — Rappresentare il Testo come Dato

## Obiettivi di Apprendimento

Al termine di questa lezione sarai in grado di:

1. **Comprendere** perché il testo deve essere convertito in numeri per il Machine Learning
2. **Applicare** tecniche di preprocessing testuale (tokenizzazione, normalizzazione)
3. **Implementare** rappresentazioni Bag of Words (BoW) e varianti
4. **Distinguere** tra rappresentazioni sparse e dense
5. **Valutare** vantaggi e limiti delle diverse rappresentazioni

## Importanza per il Data Analyst

Il testo è uno dei dati più abbondanti in azienda: email, ticket di supporto, recensioni, documenti, chat. Ma gli algoritmi di ML lavorano solo con numeri. La capacità di **trasformare testo in vettori numerici** è fondamentale per:

- Classificazione documenti
- Sentiment analysis
- Clustering di ticket/email
- Information retrieval
- Topic modeling

## Posizione nel Percorso

```
BLOCCO 4: AI & NLP
├── Lezione 29: Fondamenti AI ✓
├── Lezione 30: Rappresentare il Testo come Dato ◄── SEI QUI
├── Lezione 31: TF-IDF e Text Mining
├── Lezione 32: Sentiment Analysis
└── ...
```

---

# 1. Teoria Concettuale

## 1.1 Il Problema Fondamentale

### Perché i Numeri?

Gli algoritmi di Machine Learning operano su vettori numerici. Internamente, ogni modello esegue operazioni matematiche:

- **Distanze** (KNN, clustering): $d(x_1, x_2) = \sqrt{\sum(x_{1i} - x_{2i})^2}$
- **Prodotti scalari** (SVM, regressione): $w \cdot x + b$
- **Derivate** (gradient descent): $\frac{\partial L}{\partial w}$

Queste operazioni **non sono definite** su stringhe di testo:
- Non puoi calcolare `"ciao" + "mondo"`
- Non puoi calcolare `distanza("gatto", "cane")`
- Non puoi calcolare `derivata("documento")`

### La Trasformazione Necessaria

```
TESTO                    VETTORE NUMERICO
"Il gatto nero"    →     [0, 1, 0, 1, 0, 1, 0, ...]
"Il cane bianco"   →     [0, 0, 1, 0, 1, 0, 1, ...]
```

Questa trasformazione si chiama **vectorization** o **text embedding**.

---

## 1.2 Preprocessing del Testo

Prima di vettorizzare, il testo grezzo deve essere **preprocessato**. Questo migliora la qualità della rappresentazione.

### Pipeline di Preprocessing Tipica

```
TESTO RAW
    │
    ▼
┌─────────────────────────────────┐
│  1. LOWERCASING                 │  "Il Gatto NERO" → "il gatto nero"
└─────────────────────────────────┘
    │
    ▼
┌─────────────────────────────────┐
│  2. TOKENIZZAZIONE              │  "il gatto nero" → ["il", "gatto", "nero"]
└─────────────────────────────────┘
    │
    ▼
┌─────────────────────────────────┐
│  3. RIMOZIONE PUNTEGGIATURA     │  ["ciao", "!", "come", "?"] → ["ciao", "come"]
└─────────────────────────────────┘
    │
    ▼
┌─────────────────────────────────┐
│  4. RIMOZIONE STOPWORDS         │  ["il", "gatto", "nero"] → ["gatto", "nero"]
└─────────────────────────────────┘
    │
    ▼
┌─────────────────────────────────┐
│  5. STEMMING / LEMMATIZATION    │  ["gatti", "gattini"] → ["gatt", "gatt"]
└─────────────────────────────────┘
    │
    ▼
TOKENS PULITI
```

### Definizioni

| Operazione | Descrizione | Esempio |
|------------|-------------|---------|
| **Tokenizzazione** | Dividere il testo in unità minime (token) | "oggi piove" → ["oggi", "piove"] |
| **Stopwords** | Parole molto frequenti ma poco informative | "il", "la", "di", "che", "e", "a" |
| **Stemming** | Riduzione brutale al tema | "correndo" → "corr" |
| **Lemmatization** | Riduzione alla forma base del dizionario | "correndo" → "correre" |

### Considerazioni

Non tutte le operazioni sono sempre necessarie:
- **Sentiment analysis**: le stopwords possono essere importanti ("non" è cruciale!)
- **Named Entity Recognition**: il case è informativo ("apple" vs "Apple")
- **Document similarity**: stemming aggressivo può perdere sfumature

---

## 1.3 Bag of Words (BoW)

### L'Idea Fondamentale

Il modello **Bag of Words** rappresenta un documento come un **multi-set di parole**, ignorando l'ordine.

È chiamato "bag" (sacchetto) perché:
- Le parole vengono "buttate in un sacchetto"
- Si conta quante volte appare ogni parola
- L'ordine originale viene perso

### Costruzione del Vocabolario

**Step 1**: Raccogliere tutti i token unici dal corpus (insieme di documenti)

```
Documento 1: "il gatto mangia"
Documento 2: "il cane mangia"
Documento 3: "il gatto dorme"

Vocabolario: ["il", "gatto", "cane", "mangia", "dorme"]
             idx: 0     1       2        3        4
```

**Step 2**: Rappresentare ogni documento come vettore di conteggi

```
                    il  gatto  cane  mangia  dorme
Documento 1:       [ 1,   1,    0,     1,      0  ]
Documento 2:       [ 1,   0,    1,     1,      0  ]
Documento 3:       [ 1,   1,    0,     0,      1  ]
```

### Formalizzazione

Dato un vocabolario $V = \{w_1, w_2, ..., w_n\}$, un documento $d$ viene rappresentato come:

$$\vec{d} = [c(w_1, d), c(w_2, d), ..., c(w_n, d)]$$

Dove $c(w_i, d)$ = numero di occorrenze della parola $w_i$ nel documento $d$.

### Variante: Binary BoW

Invece di contare, si usa 0/1 (presenza/assenza):

$$\vec{d} = [\mathbb{1}(w_1 \in d), \mathbb{1}(w_2 \in d), ..., \mathbb{1}(w_n \in d)]$$

Dove $\mathbb{1}(condizione)$ = 1 se vera, 0 altrimenti.

---

## 1.4 N-grammi

### Limitazione del BoW

Il BoW perde completamente l'ordine delle parole. Questo può essere problematico:

| Frase | BoW | Significato |
|-------|-----|-------------|
| "Il gatto mangia il topo" | {gatto:1, mangia:1, topo:1} | Il gatto è predatore |
| "Il topo mangia il gatto" | {gatto:1, mangia:1, topo:1} | Il topo è predatore |

**Stesso vettore, significato opposto!**

### Soluzione: N-grammi

Un **n-gramma** è una sequenza contigua di n elementi.

| n | Nome | Esempio per "il gatto nero" |
|---|------|----------------------------|
| 1 | Unigram | ["il", "gatto", "nero"] |
| 2 | Bigram | ["il gatto", "gatto nero"] |
| 3 | Trigram | ["il gatto nero"] |

### Esempio Pratico

```
Frase: "non mi piace"

Unigrammi:  ["non", "mi", "piace"]
Bigrammi:   ["non mi", "mi piace"]
Trigrammi:  ["non mi piace"]

Combinato (1,2): ["non", "mi", "piace", "non mi", "mi piace"]
```

### Trade-off

| Aspetto | Unigrammi | Bigrammi/Trigrammi |
|---------|-----------|-------------------|
| **Dimensionalità** | Moderata | Alta (esponenziale) |
| **Cattura ordine** | No | Parzialmente |
| **Sparsità** | Moderata | Molto alta |
| **Generalizzazione** | Buona | Può essere limitata |

Tipicamente si usa un mix: `ngram_range=(1,2)` cattura sia parole singole che coppie.

---

## 1.5 Rappresentazioni Sparse vs Dense

### Caratteristiche delle Rappresentazioni

Le rappresentazioni testuali possono essere:

**SPARSE (BoW, TF-IDF)**
```
Vettore: [0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, ...]
          ↑  ↑  ↑     ↑  ↑  ↑     ↑  ↑  ↑  ↑  ↑     ↑
        Molti zeri, pochi valori non-nulli
        
Dimensionalità: 10.000 - 100.000+ (dimensione vocabolario)
```

**DENSE (Word2Vec, BERT embeddings)**
```
Vettore: [0.23, -0.45, 0.12, 0.78, -0.33, 0.56, ...]
          ↑      ↑      ↑      ↑      ↑      ↑
        Tutti valori non-nulli, significativi
        
Dimensionalità: 100 - 1024 (fissa, compatta)
```

### Confronto

| Aspetto | Sparse (BoW) | Dense (Embeddings) |
|---------|--------------|-------------------|
| **Dimensionalità** | Alta (~vocabolario) | Bassa (100-1024) |
| **Interpretabilità** | Alta (ogni dim = parola) | Bassa (dimensioni astratte) |
| **Similarità semantica** | Limitata | Catturata |
| **Out-of-vocabulary** | Ignorato | Gestibile |
| **Complessità** | Semplice | Richiede pre-training |
| **Storage** | Efficiente (CSR) | Denso ma compatto |

### Esempio di Similarità

**Sparse (BoW)**:
```
"re"    → [0, 0, 1, 0, 0, 0, ...]  (indice di "re")
"regina"→ [0, 0, 0, 0, 1, 0, ...]  (indice di "regina")

Similarità coseno = 0 (vettori ortogonali!)
```

**Dense (Word2Vec)**:
```
"re"    → [0.23, 0.45, -0.12, ...]
"regina"→ [0.25, 0.42, -0.10, ...]

Similarità coseno ≈ 0.95 (vettori molto simili)
```

### Quando Usare Cosa

| Scenario | Consigliato |
|----------|-------------|
| Classificazione documenti (molti dati) | BoW/TF-IDF |
| Similarità semantica | Dense embeddings |
| Interpretabilità richiesta | BoW (ogni feature = parola) |
| Risorse limitate | BoW (nessun modello pre-trained) |
| Transfer learning | Dense embeddings pre-trained |

---

# 2. Schema Mentale

## Mappa Decisionale: Vettorizzazione del Testo

```
                    ┌─────────────────────────┐
                    │    HAI TESTO DA         │
                    │    PROCESSARE?          │
                    └────────────┬────────────┘
                                 │
                    ┌────────────▼────────────┐
                    │   PREPROCESSING         │
                    │   (tokenize, clean)     │
                    └────────────┬────────────┘
                                 │
              ┌──────────────────┼──────────────────┐
              │                  │                  │
              ▼                  ▼                  ▼
    ┌─────────────────┐ ┌─────────────────┐ ┌─────────────────┐
    │  Bag of Words   │ │  TF-IDF         │ │  Embeddings     │
    │  (CountVec)     │ │  (TfidfVec)     │ │  (Word2Vec, etc)│
    └────────┬────────┘ └────────┬────────┘ └────────┬────────┘
             │                   │                   │
             │                   │                   │
    ┌────────▼────────┐ ┌────────▼────────┐ ┌────────▼────────┐
    │ SPARSE          │ │ SPARSE          │ │ DENSE           │
    │ Interpretabile  │ │ Pesato per      │ │ Semanticamente  │
    │ Conta frequenze │ │ importanza      │ │ ricco           │
    └─────────────────┘ └─────────────────┘ └─────────────────┘
```

## Checklist Preprocessing

```
□ Lowercase? 
  → Sì per la maggior parte dei task
  → No se case è informativo (NER, formal vs informal)

□ Rimuovere punteggiatura?
  → Sì solitamente
  → No se emoji/emoticon sono informative (sentiment)

□ Rimuovere stopwords?
  → Sì per topic modeling, similarity
  → No per sentiment ("non" è cruciale), sequenze

□ Stemming/Lemmatization?
  → Stemming: veloce, aggressivo, può perdere info
  → Lemmatization: più accurato, più lento
  → Nessuno: se le forme flesse sono informative
```

## Parametri Chiave CountVectorizer

| Parametro | Significato | Default |
|-----------|-------------|---------|
| `max_features` | Limita vocabolario alle top N parole | None (tutte) |
| `ngram_range` | Range di n-grammi (min, max) | (1,1) unigram |
| `min_df` | Ignora termini in meno di X documenti | 1 |
| `max_df` | Ignora termini in più del X% documenti | 1.0 |
| `binary` | 0/1 invece di conteggi | False |
| `stop_words` | Lista di stopwords da rimuovere | None |

---

# 3. Notebook Dimostrativo

## Rappresentazione del Testo in Pratica

Dimostriamo step-by-step come trasformare testo in vettori numerici utilizzabili per ML.

In [ ]:
# ============================================================
# SETUP: Importazione librerie per NLP
# ============================================================

import numpy as np                          # Array numerici
import pandas as pd                         # DataFrames
from sklearn.feature_extraction.text import CountVectorizer  # Bag of Words
from sklearn.feature_extraction.text import TfidfVectorizer  # TF-IDF (preview)
import re                                   # Regular expressions per pulizia
import warnings
warnings.filterwarnings('ignore')

# Seed per riproducibilità
np.random.seed(42)

print("Librerie importate correttamente")

In [ ]:
# ============================================================
# DATASET: Corpus di esempio (recensioni prodotto)
# ============================================================

# Creiamo un piccolo corpus di recensioni per dimostrazione
corpus = [
    "Ottimo prodotto, qualità eccellente!",
    "Prodotto scadente, non lo consiglio",
    "Qualità buona, prezzo giusto",
    "Non funziona, prodotto difettoso",
    "Consiglio questo prodotto, ottimo acquisto",
    "Prezzo alto ma qualità ottima",
    "Prodotto arrivato rotto, pessima esperienza",
    "Acquisto consigliato, spedizione veloce"
]

print("=" * 60)
print("CORPUS DI ESEMPIO")
print("=" * 60)
for i, doc in enumerate(corpus, 1):
    print(f"Doc {i}: \"{doc}\"")

In [ ]:
# ============================================================
# STEP 1: PREPROCESSING MANUALE
# ============================================================
# Dimostriamo ogni step del preprocessing

def preprocess_text(text: str) -> str:
    """
    Applica preprocessing di base a un testo.
    
    Steps:
    1. Lowercase
    2. Rimozione punteggiatura
    3. Rimozione numeri
    4. Rimozione spazi multipli
    """
    # Step 1: Tutto minuscolo
    text = text.lower()
    
    # Step 2: Rimuovi punteggiatura (sostituisci con spazio)
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Step 3: Rimuovi numeri
    text = re.sub(r'\d+', '', text)
    
    # Step 4: Rimuovi spazi multipli
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

print("=" * 60)
print("PREPROCESSING MANUALE")
print("=" * 60)

print("\nTrasformazione testo grezzo → testo pulito:")
print("-" * 60)

for i, doc in enumerate(corpus[:3], 1):
    cleaned = preprocess_text(doc)
    print(f"\nDoc {i}:")
    print(f"  Originale: \"{doc}\"")
    print(f"  Pulito:    \"{cleaned}\"")

In [ ]:
# ============================================================
# STEP 2: TOKENIZZAZIONE
# ============================================================
# Dividiamo il testo in token (parole)

def tokenize(text: str) -> list:
    """
    Tokenizza un testo in lista di parole.
    Versione semplice: split su spazi.
    """
    return text.split()

print("=" * 60)
print("TOKENIZZAZIONE")
print("=" * 60)

print("\nDivisione in token:")
print("-" * 60)

for i, doc in enumerate(corpus[:3], 1):
    cleaned = preprocess_text(doc)
    tokens = tokenize(cleaned)
    print(f"\nDoc {i}:")
    print(f"  Testo pulito: \"{cleaned}\"")
    print(f"  Tokens: {tokens}")
    print(f"  N. tokens: {len(tokens)}")

In [ ]:
# ============================================================
# STEP 3: BAG OF WORDS con CountVectorizer
# ============================================================
# sklearn gestisce preprocessing + tokenizzazione + BoW

# Creiamo il vectorizer con configurazione base
vectorizer = CountVectorizer()

# Fit: costruisce il vocabolario
# Transform: converte i documenti in vettori
X_bow = vectorizer.fit_transform(corpus)

print("=" * 60)
print("BAG OF WORDS - CountVectorizer")
print("=" * 60)

# Vocabolario appreso
print("\n1. VOCABOLARIO APPRESO")
print("-" * 40)
vocab = vectorizer.get_feature_names_out()
print(f"Dimensione vocabolario: {len(vocab)} parole")
print(f"Parole: {list(vocab)}")

# Matrice documento-termine
print("\n2. MATRICE DOCUMENTO-TERMINE")
print("-" * 40)
print(f"Shape: {X_bow.shape}")
print(f"→ {X_bow.shape[0]} documenti x {X_bow.shape[1]} features (parole)")
print(f"Tipo: {type(X_bow)} (matrice sparsa)")

# Convertiamo in DataFrame per visualizzazione
df_bow = pd.DataFrame(
    X_bow.toarray(),                    # .toarray() converte sparse → dense
    columns=vocab,
    index=[f"Doc {i+1}" for i in range(len(corpus))]
)

print("\n3. VISUALIZZAZIONE (prime 5 colonne)")
print("-" * 40)
print(df_bow.iloc[:, :8])

In [ ]:
# ============================================================
# ANALISI: Sparsità della matrice
# ============================================================

print("=" * 60)
print("ANALISI DELLA SPARSITÀ")
print("=" * 60)

# Calcolo sparsità
n_total = X_bow.shape[0] * X_bow.shape[1]  # Elementi totali
n_nonzero = X_bow.nnz                       # Elementi non-zero
sparsity = 1 - (n_nonzero / n_total)

print(f"\nElementi totali nella matrice: {n_total}")
print(f"Elementi non-zero: {n_nonzero}")
print(f"Elementi zero: {n_total - n_nonzero}")
print(f"Sparsità: {sparsity:.1%}")

print(f"\n→ Solo il {100-sparsity*100:.1f}% della matrice contiene informazione utile")
print("→ Con vocabolari grandi (10k+ parole), la sparsità supera il 99%")

# Visualizziamo la matrice come heatmap testuale
print("\n" + "=" * 60)
print("PATTERN DELLA MATRICE (0 = assente, ≥1 = presente)")
print("=" * 60)
print()
# Mostriamo presenza/assenza
for idx, row in df_bow.iterrows():
    pattern = "".join(["█" if v > 0 else "░" for v in row])
    n_words = (row > 0).sum()
    print(f"{idx}: {pattern}  ({n_words} parole uniche)")

In [ ]:
# ============================================================
# N-GRAMMI: Catturare sequenze di parole
# ============================================================

# Vectorizer con bigrammi
vec_bigram = CountVectorizer(ngram_range=(1, 2))  # Unigram + bigram
X_bigram = vec_bigram.fit_transform(corpus)

print("=" * 60)
print("N-GRAMMI: UNIGRAM + BIGRAM")
print("=" * 60)

vocab_bigram = vec_bigram.get_feature_names_out()

print(f"\nDimensione vocabolario (solo unigram): {len(vocab)} parole")
print(f"Dimensione vocabolario (uni+bigram): {len(vocab_bigram)} features")
print(f"→ Aumento: +{len(vocab_bigram) - len(vocab)} features ({(len(vocab_bigram)/len(vocab)-1)*100:.0f}% in più)")

print("\n" + "-" * 60)
print("ESEMPI DI BIGRAMMI ESTRATTI:")
print("-" * 60)

# Filtriamo solo i bigrammi (contengono spazio)
bigrams = [f for f in vocab_bigram if ' ' in f]
print(f"\nBigrammi trovati ({len(bigrams)}):")
for bg in bigrams[:15]:
    print(f"  • \"{bg}\"")

In [ ]:
# ============================================================
# PARAMETRI UTILI DI CountVectorizer
# ============================================================

print("=" * 60)
print("PARAMETRI CountVectorizer: ESEMPI PRATICI")
print("=" * 60)

# 1. max_features: limita vocabolario
print("\n1. max_features=10 (solo le 10 parole più frequenti)")
print("-" * 50)
vec_limited = CountVectorizer(max_features=10)
X_limited = vec_limited.fit_transform(corpus)
print(f"Vocabolario: {list(vec_limited.get_feature_names_out())}")

# 2. min_df: ignora parole rare
print("\n2. min_df=2 (parole in almeno 2 documenti)")
print("-" * 50)
vec_mindf = CountVectorizer(min_df=2)
X_mindf = vec_mindf.fit_transform(corpus)
print(f"Vocabolario ridotto: {list(vec_mindf.get_feature_names_out())}")
print(f"Parole rimosse: {len(vocab) - len(vec_mindf.get_feature_names_out())}")

# 3. max_df: ignora parole troppo comuni
print("\n3. max_df=0.5 (parole in max 50% documenti)")
print("-" * 50)
vec_maxdf = CountVectorizer(max_df=0.5)
X_maxdf = vec_maxdf.fit_transform(corpus)
print(f"Vocabolario: {list(vec_maxdf.get_feature_names_out())}")

# 4. binary: presenza/assenza invece di conteggi
print("\n4. binary=True (0/1 invece di conteggi)")
print("-" * 50)
vec_binary = CountVectorizer(binary=True)
X_binary = vec_binary.fit_transform(corpus)
# Esempio con documento che ripete una parola
doc_repeat = ["buono buono buono ottimo"]
X_count = CountVectorizer().fit_transform(doc_repeat)
X_bin = CountVectorizer(binary=True).fit_transform(doc_repeat)
print(f"Documento: \"{doc_repeat[0]}\"")
print(f"Con conteggi: {X_count.toarray()[0]}")
print(f"Con binary:   {X_bin.toarray()[0]}")

In [ ]:
# ============================================================
# TRASFORMAZIONE DI NUOVI DOCUMENTI
# ============================================================
# Importante: usare .transform() (non fit_transform) per nuovi dati

print("=" * 60)
print("TRASFORMARE NUOVI DOCUMENTI")
print("=" * 60)

# Vectorizer già addestrato (su corpus)
print("\nVectorizer addestrato sul corpus originale")
print(f"Vocabolario: {list(vectorizer.get_feature_names_out())}")

# Nuovo documento da trasformare
nuovo_doc = ["Prodotto eccellente, prezzo ottimo"]
print(f"\nNuovo documento: \"{nuovo_doc[0]}\"")

# CORRETTO: usare transform (non fit_transform)
X_nuovo = vectorizer.transform(nuovo_doc)

print(f"\nVettore risultante:")
print(f"  Shape: {X_nuovo.shape}")
print(f"  Valori: {X_nuovo.toarray()[0]}")

# Quali parole sono state riconosciute?
parole_riconosciute = []
for i, count in enumerate(X_nuovo.toarray()[0]):
    if count > 0:
        parole_riconosciute.append(vectorizer.get_feature_names_out()[i])

print(f"\nParole riconosciute dal vocabolario: {parole_riconosciute}")

# Parole nuove (out-of-vocabulary) vengono ignorate
print("\n⚠️  NOTA: 'eccellente' non era nel vocabolario → IGNORATA")
print("    Questo è un limite del BoW: parole nuove scompaiono")

---

# 4. Esercizi Svolti

## Esercizio 1: Preprocessing Pipeline

**Problema**: Implementa una pipeline completa di preprocessing che:
1. Converta in lowercase
2. Rimuova punteggiatura e numeri
3. Tokenizzi
4. Rimuova le stopwords italiane
5. Restituisca i token puliti

In [ ]:
# ============================================================
# ESERCIZIO 1 - SOLUZIONE
# Pipeline completa di preprocessing testuale
# ============================================================

# Stopwords italiane comuni
STOPWORDS_IT = {
    'il', 'lo', 'la', 'i', 'gli', 'le', 'un', 'uno', 'una',
    'di', 'a', 'da', 'in', 'con', 'su', 'per', 'tra', 'fra',
    'e', 'o', 'ma', 'che', 'non', 'è', 'sono', 'ha', 'ho',
    'questo', 'questa', 'questi', 'queste', 'quello', 'quella',
    'come', 'quando', 'dove', 'perché', 'chi', 'cosa',
    'più', 'molto', 'anche', 'solo', 'già', 'ancora', 'sempre'
}

def full_preprocessing_pipeline(text: str, remove_stopwords: bool = True) -> list:
    """
    Pipeline completa di preprocessing testuale.
    
    Parameters:
    -----------
    text : str
        Testo da preprocessare
    remove_stopwords : bool
        Se True, rimuove le stopwords italiane
    
    Returns:
    --------
    list : Lista di token puliti
    
    Steps:
    1. Lowercase
    2. Rimozione punteggiatura e numeri
    3. Tokenizzazione
    4. Rimozione stopwords (opzionale)
    5. Rimozione token vuoti/corti
    """
    
    # Step 1: Lowercase
    text = text.lower()
    
    # Step 2: Rimozione punteggiatura e numeri
    # Manteniamo solo lettere e spazi
    text = re.sub(r'[^a-zàèéìòù\s]', ' ', text)
    
    # Step 3: Tokenizzazione
    tokens = text.split()
    
    # Step 4: Rimozione stopwords (se richiesto)
    if remove_stopwords:
        tokens = [t for t in tokens if t not in STOPWORDS_IT]
    
    # Step 5: Rimozione token troppo corti (1 carattere)
    tokens = [t for t in tokens if len(t) > 1]
    
    return tokens

# ── TEST DELLA PIPELINE ──
print("=" * 70)
print("ESERCIZIO 1: PREPROCESSING PIPELINE")
print("=" * 70)

testi_test = [
    "Il prodotto è OTTIMO! Costa solo 29.99€ ma vale molto di più.",
    "Non mi piace questo servizio... 3 stelle su 5.",
    "Consegna veloce, il pacco è arrivato in 2 giorni!!!"
]

for i, testo in enumerate(testi_test, 1):
    print(f"\n{'─' * 70}")
    print(f"TESTO {i}")
    print(f"{'─' * 70}")
    print(f"Originale: \"{testo}\"")
    
    tokens_con_sw = full_preprocessing_pipeline(testo, remove_stopwords=False)
    tokens_senza_sw = full_preprocessing_pipeline(testo, remove_stopwords=True)
    
    print(f"\nCon stopwords:    {tokens_con_sw}")
    print(f"Senza stopwords:  {tokens_senza_sw}")
    print(f"Stopwords rimosse: {set(tokens_con_sw) - set(tokens_senza_sw)}")

---

## Esercizio 2: Costruzione Manuale BoW

**Problema**: Costruisci "a mano" (senza CountVectorizer) la matrice Bag of Words per un piccolo corpus, per capire esattamente cosa fa CountVectorizer internamente.

In [ ]:
# ============================================================
# ESERCIZIO 2 - SOLUZIONE
# Costruzione manuale della matrice Bag of Words
# ============================================================

def manual_bag_of_words(corpus: list) -> tuple:
    """
    Costruisce manualmente la matrice BoW.
    
    Parameters:
    -----------
    corpus : list
        Lista di documenti (stringhe)
    
    Returns:
    --------
    tuple : (matrice BoW come numpy array, vocabolario come lista)
    """
    
    # Step 1: Preprocessa e tokenizza ogni documento
    tokenized_docs = []
    for doc in corpus:
        # Semplice preprocessing
        doc_lower = doc.lower()
        doc_clean = re.sub(r'[^\w\s]', '', doc_lower)
        tokens = doc_clean.split()
        tokenized_docs.append(tokens)
    
    # Step 2: Costruisci vocabolario (insieme di tutte le parole uniche)
    all_words = set()
    for tokens in tokenized_docs:
        all_words.update(tokens)
    
    # Step 3: Ordina alfabeticamente (come fa CountVectorizer)
    vocabulary = sorted(list(all_words))
    
    # Step 4: Crea mapping parola → indice
    word_to_idx = {word: idx for idx, word in enumerate(vocabulary)}
    
    # Step 5: Costruisci matrice
    n_docs = len(corpus)
    n_words = len(vocabulary)
    bow_matrix = np.zeros((n_docs, n_words), dtype=int)
    
    for doc_idx, tokens in enumerate(tokenized_docs):
        for token in tokens:
            word_idx = word_to_idx[token]
            bow_matrix[doc_idx, word_idx] += 1
    
    return bow_matrix, vocabulary

# ── TEST E CONFRONTO ──
print("=" * 70)
print("ESERCIZIO 2: COSTRUZIONE MANUALE BoW")
print("=" * 70)

# Corpus semplice per test
mini_corpus = [
    "il gatto mangia il pesce",
    "il cane gioca",
    "il gatto gioca con il cane"
]

print("\nCORPUS:")
for i, doc in enumerate(mini_corpus, 1):
    print(f"  Doc {i}: \"{doc}\"")

# Costruzione manuale
bow_manual, vocab_manual = manual_bag_of_words(mini_corpus)

print("\n" + "-" * 70)
print("RISULTATO MANUALE:")
print("-" * 70)
print(f"\nVocabolario: {vocab_manual}")
print(f"\nMatrice BoW:")
df_manual = pd.DataFrame(bow_manual, columns=vocab_manual, 
                         index=[f"Doc {i}" for i in range(1, 4)])
print(df_manual)

# Confronto con CountVectorizer
print("\n" + "-" * 70)
print("CONFRONTO CON CountVectorizer:")
print("-" * 70)
cv = CountVectorizer()
bow_sklearn = cv.fit_transform(mini_corpus)
vocab_sklearn = list(cv.get_feature_names_out())

print(f"\nVocabolario sklearn: {vocab_sklearn}")
print(f"\nMatrici identiche? {np.array_equal(bow_manual, bow_sklearn.toarray())}")
print("✓ La nostra implementazione produce lo stesso risultato di sklearn!")

---

## Esercizio 3: Classificazione con BoW

**Problema**: Usa la rappresentazione BoW per classificare recensioni come positive o negative (mini sentiment analysis).

In [ ]:
# ============================================================
# ESERCIZIO 3 - SOLUZIONE
# Classificazione di sentiment con BoW
# ============================================================

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

print("=" * 70)
print("ESERCIZIO 3: CLASSIFICAZIONE SENTIMENT CON BoW")
print("=" * 70)

# ── STEP 1: Dataset di recensioni etichettate ──
recensioni = [
    # Positive (1)
    ("Prodotto eccezionale, lo consiglio a tutti", 1),
    ("Ottimo acquisto, sono molto soddisfatto", 1),
    ("Qualità superiore, prezzo giusto", 1),
    ("Spedizione veloce, prodotto perfetto", 1),
    ("Fantastico, supera le aspettative", 1),
    ("Molto buono, lo ricomprerei", 1),
    ("Ottima qualità, consigliato", 1),
    ("Prodotto top, eccellente", 1),
    ("Bellissimo prodotto, funziona benissimo", 1),
    ("Soddisfatto dell'acquisto, ottimo", 1),
    
    # Negative (0)
    ("Prodotto scadente, non funziona", 0),
    ("Pessima qualità, soldi sprecati", 0),
    ("Non lo consiglio, deludente", 0),
    ("Arrivato rotto, pessimo servizio", 0),
    ("Qualità scarsa, non vale il prezzo", 0),
    ("Terribile, da evitare", 0),
    ("Prodotto difettoso, reso immediato", 0),
    ("Brutta esperienza, non comprate", 0),
    ("Deluso totalmente, prodotto inutile", 0),
    ("Pessimo acquisto, sconsigliato", 0)
]

# Separiamo testi e labels
texts = [r[0] for r in recensioni]
labels = [r[1] for r in recensioni]

print(f"\nDataset: {len(texts)} recensioni")
print(f"  Positive: {sum(labels)}")
print(f"  Negative: {len(labels) - sum(labels)}")

# ── STEP 2: Vettorizzazione ──
print("\n" + "-" * 70)
print("STEP 2: VETTORIZZAZIONE BOW")
print("-" * 70)

vectorizer_sent = CountVectorizer(max_features=100)
X = vectorizer_sent.fit_transform(texts)
y = np.array(labels)

print(f"Matrice features: {X.shape}")
print(f"Vocabolario: {len(vectorizer_sent.get_feature_names_out())} parole")

# ── STEP 3: Split train/test ──
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"\nTrain: {X_train.shape[0]} campioni")
print(f"Test: {X_test.shape[0]} campioni")

# ── STEP 4: Training classificatore ──
print("\n" + "-" * 70)
print("STEP 4: TRAINING NAIVE BAYES")
print("-" * 70)

# Naive Bayes è particolarmente adatto per text classification
clf = MultinomialNB()
clf.fit(X_train, y_train)

# ── STEP 5: Valutazione ──
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"\nAccuracy sul test set: {accuracy:.2%}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Negativo', 'Positivo']))

# ── STEP 6: Test su nuove recensioni ──
print("-" * 70)
print("TEST SU NUOVE RECENSIONI:")
print("-" * 70)

nuove_recensioni = [
    "Prodotto buono, mi piace molto",
    "Terribile esperienza, non comprate mai",
    "Niente di speciale, nella media"
]

for rec in nuove_recensioni:
    X_new = vectorizer_sent.transform([rec])
    pred = clf.predict(X_new)[0]
    prob = clf.predict_proba(X_new)[0]
    
    sentiment = "POSITIVO" if pred == 1 else "NEGATIVO"
    conf = max(prob) * 100
    
    print(f"\n\"{rec}\"")
    print(f"  → Predizione: {sentiment} (confidence: {conf:.0f}%)")

---

# 5. Conclusione Operativa

## Cosa Abbiamo Imparato

| Concetto | Implicazione Pratica |
|----------|---------------------|
| **Testo → Numeri** | Gli algoritmi ML richiedono input numerici; il testo va vettorizzato |
| **Preprocessing** | Pulizia del testo (lowercase, punteggiatura, stopwords) migliora la qualità |
| **Bag of Words** | Rappresenta documenti come conteggi di parole, ignora l'ordine |
| **N-grammi** | Catturano sequenze di parole, aumentano dimensionalità |
| **Sparse vs Dense** | BoW produce vettori sparsi; embeddings producono vettori densi |
| **CountVectorizer** | Strumento sklearn per BoW, con molti parametri utili |

## Workflow Operativo

```
1. ESPLORAZIONE
   └── Analizza un campione di testi (lunghezza, lingua, rumore)

2. PREPROCESSING
   ├── Decidi: lowercase? stopwords? stemming?
   └── Implementa pipeline coerente train/test

3. VETTORIZZAZIONE
   ├── Scegli metodo (BoW, TF-IDF, embeddings)
   ├── Configura parametri (max_features, ngram_range)
   └── fit_transform su train, transform su test

4. MODELLAZIONE
   └── Usa la matrice risultante come input per classificatori
```

## Errori Comuni da Evitare

1. ❌ Usare `fit_transform` anche sui dati di test (data leakage)
2. ❌ Ignorare parole out-of-vocabulary senza rendersene conto
3. ❌ Applicare preprocessing diverso tra train e inference
4. ❌ Non considerare la sparsità della matrice risultante
5. ❌ Rimuovere stopwords quando sono informative (es. sentiment)

## Prossimi Passi

- **Lezione 31**: TF-IDF — Pesare le parole per importanza
- **Lezione 32**: Sentiment Analysis — Applicazione pratica completa

---

# 6. Bignami — Scheda di Riferimento Rapido

## Pipeline di Preprocessing

| Step | Operazione | Esempio |
|------|------------|---------|
| 1 | Lowercase | "CIAO" → "ciao" |
| 2 | Rimuovi punteggiatura | "ciao!" → "ciao" |
| 3 | Tokenizza | "il gatto" → ["il", "gatto"] |
| 4 | Rimuovi stopwords | ["il", "gatto"] → ["gatto"] |
| 5 | Stemming/Lemma | ["gatti"] → ["gatt"] |

## Rappresentazioni Testuali

| Metodo | Tipo | Dimensionalità | Pro | Contro |
|--------|------|----------------|-----|--------|
| **BoW** | Sparse | Alta (~vocabolario) | Semplice, interpretabile | Perde ordine |
| **TF-IDF** | Sparse | Alta | Pesa importanza | Perde ordine |
| **Word2Vec** | Dense | Bassa (100-300) | Semantica | Richiede training |
| **BERT** | Dense | Media (768) | Contestuale | Computazionalmente pesante |

## Codice Essenziale

```python
from sklearn.feature_extraction.text import CountVectorizer

# Configurazione tipica
vectorizer = CountVectorizer(
    max_features=10000,    # Limita vocabolario
    ngram_range=(1, 2),    # Unigram + bigram
    min_df=2,              # Ignora parole rare
    max_df=0.95,           # Ignora parole troppo comuni
    stop_words=None        # O lista custom
)

# Training: costruisce vocabolario
X_train = vectorizer.fit_transform(train_texts)

# Inference: usa vocabolario esistente
X_test = vectorizer.transform(test_texts)

# Ispeziona vocabolario
vocab = vectorizer.get_feature_names_out()
```

## Checklist Pre-Vettorizzazione

```
□ Il preprocessing è coerente tra train e test?
□ Ho considerato se le stopwords sono informative?
□ Ho scelto max_features appropriato per la RAM disponibile?
□ Ho verificato la sparsità risultante?
□ Uso transform() (non fit_transform) sui nuovi dati?
```

---
*Fine Lezione 30 — Rappresentare il Testo come Dato*